# Zusammenhang zwischen Covid-19, Bewegungsdaten und dem Wetter

Aufbauend auf das vorherige Notebook wird hier ein Zusammenhang zwischen Bewegungsdaten, Covid-19 und dem Wetter gesucht.
Hypothese: Bei schlechterem Wetter verlassen weniger Menschen ihr Haus, sie treffen dadurch auf weniger Menschen, die Ansteckungsrate sinkt.

Ermittelt wird dies ebenfalls am Beispiel New York.
Als Bewegungsdaten werden Daten herangezogen, die eine Abweichung der Anzahl von "zu Hause Bleibenden" im Gegensatz zum Normalwert angeben.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from pathlib import Path
import plotly.offline as py
import plotly.express as px
import cufflinks as cf
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.cluster import DBSCAN

# USA
Die Stadt New York City steht weltweit im Fokus, weil es hier die meisten Infizierten in den USA momentan gibt.

In [ ]:
#einlesen der notwenidigen Daten
USA = pd.read_csv('/kaggle/input/covid19-in-usa/us_states_covid19_daily.csv')
WetterNY = pd.read_csv('/kaggle/input/wetter/NewYork_all2.csv',sep = ";")
bewegungny = pd.read_csv('/kaggle/input/bewegungny/bewegungny2.csv',sep = ";")

In [ ]:
#USA.head()

In [ ]:
#USA.info()

In [ ]:
#WetterNY.head()

In [ ]:
#WetterNY.info()

In [ ]:
#bewegungny

In [ ]:
# Zeilen mit Null Werten löschen

#bewegungny.dropna()

In [ ]:
#bewegungny['date'] = bewegungny['date'].astype(int)

# Aufsplittung der Daten in die verschiedenen Bundesstaaten der USA

In [ ]:
#USA['state'].unique()

In [ ]:
NYC = USA[USA.state=='NY']
#NYC.head()

In [ ]:
#NYC.info()

In [ ]:
#NYC = NYC[NYC['positive']>0]
#NYC
#New York nur die positiven Fälle

In [ ]:
#Wetterdaten und Coronadaten zusammenführen

#coronawetterny = pd.merge(NYC, WetterNY, how='right', on='date')
coronawetterny = pd.merge(WetterNY, NYC, how='inner', on='date')
coronawetterny.head()

In [ ]:
coronawetterny.set_index('date')

In [ ]:
coronawetterny2 = coronawetterny.copy()
coronawetterny2=coronawetterny2.drop(columns=['pending','hospitalizedCurrently', 'negative', 'hospitalizedCumulative', 'inIcuCurrently','inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative', 'lastUpdateEt', 'hash', 'recovered', 'dataQualityGrade', 'dateChecked', 'death', 'hospitalized', 'totalTestResults', 'total', 'posNeg', 'fips', 'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'totalTestResultsIncrease'])


In [ ]:
#coronawetterny2.info()

In [ ]:
coronawetterny2['date'] = coronawetterny2['date'].astype(str)

In [ ]:
coronawetterny2.set_index('date', drop=True,inplace=True)
coronawetterny2.head()

In [ ]:
coronawetterny2.corr(method ='kendall')
#korrelation zwischen -1 und 1
#  1 -> korreliert
#  0 -> kein Zusammenhang
# -1 -> korreliert entgegengesetzt

In [ ]:
for i in range(14):
    i=i-14
    coronawetterny2.loc[:,('positiveIncrease ' + str(i) + ' days after weather')] = coronawetterny2.positiveIncrease.shift(i)

In [ ]:
coronawetterny2

In [ ]:
korrmat=coronawetterny2.corr(method ='kendall')

In [ ]:
korrmat

In [ ]:
korrmatcol=korrmat.copy()
korrmatcol=korrmatcol.drop(columns=['positiveIncrease -1 days after weather','positiveIncrease -2 days after weather', 'positiveIncrease -3 days after weather', 'positiveIncrease -5 days after weather', 'positiveIncrease -4 days after weather', 'positiveIncrease -6 days after weather', 'positiveIncrease -7 days after weather', 'positiveIncrease -8 days after weather', 'positiveIncrease -9 days after weather','positiveIncrease -10 days after weather','positiveIncrease -11 days after weather','positiveIncrease -12 days after weather','positiveIncrease -13 days after weather','positiveIncrease -14 days after weather', 'positive', 'positiveIncrease'])


In [ ]:
korrmatcol.drop(['Avg Temperature in C', 'Avg Dew Point in C', 'Avg Humidity in Percent', 'Avg Wind Speed in mph', 'Avg Pressure in Hg', 'Precipitation' ])

In [ ]:
wetterkorr=WetterNY.corr(method ='kendall')
wetterkorr['Avg Pressure in Hg']

# Zusammenhang Bewegung und Corona

In [ ]:
# BEwegungsdaten und Coronadaten in neuem DF zusammenführen

coronabewegungny = pd.merge(bewegungny, NYC, how='inner', on='date')
coronabewegungny

In [ ]:
#nicht benötigte Spalten löschen

coronabewegungny2 = coronabewegungny.copy()
coronabewegungny2=coronabewegungny2.drop(columns=['pending','hospitalizedCurrently', 'negative', 'hospitalizedCumulative', 'inIcuCurrently','inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative', 'lastUpdateEt', 'hash', 'recovered', 'dataQualityGrade', 'dateChecked', 'death', 'hospitalized', 'totalTestResults', 'total', 'posNeg', 'fips', 'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'totalTestResultsIncrease'])
coronabewegungny2

In [ ]:
# date als String format

coronabewegungny2['date'] = coronabewegungny2['date'].astype(str)

In [ ]:
# date als index setzen

coronabewegungny2.set_index('date', drop=True,inplace=True)
coronabewegungny2

In [ ]:
#korrelation der bewegung mit neuinfizierten

korrbew=coronabewegungny2['zuhause abweichung baseline in prozent'].corr(coronabewegungny2['positiveIncrease'])
korrbew


die matrix ist nicht aussagekräftig. die aussage wäre, dass je mehr menschen zu hause bleiben, desto mehr werden angesteckt.
durch die höhere ansteckungsrate werden allerdings strengere restriktionen geltend, dadurch bleiben mehr menschen zu hause.
die hier gefundene korrelation ist nicht relevant.

In [ ]:
pd.set_option('display.max_rows', coronabewegungny2.shape[0]+1)
coronabewegungny2

In [ ]:
#schleift neuerkrankungen 14 tage durch
for i in range(14):
    i=i-14
    coronabewegungny2.loc[:,('positiveIncrease ' + str(i) + ' days after staying home')] = coronabewegungny2.positiveIncrease.shift(i)

In [ ]:
coronabewegungny2

In [ ]:
#korrelationsmatrix mit allen daten
korrbewall=coronabewegungny2.corr(method ='kendall')
korrbewall

In [ ]:
#nicht benötigte spalten löschen
korrbewallcol=korrbewall.copy()
korrbewallcol=korrbewallcol.drop(columns=['positiveIncrease -1 days after staying home','positiveIncrease -2 days after staying home', 'positiveIncrease -3 days after staying home', 'positiveIncrease -5 days after staying home', 'positiveIncrease -4 days after staying home', 'positiveIncrease -6 days after staying home', 'positiveIncrease -7 days after staying home', 'positiveIncrease -8 days after staying home', 'positiveIncrease -9 days after staying home','positiveIncrease -10 days after staying home','positiveIncrease -11 days after staying home','positiveIncrease -12 days after staying home','positiveIncrease -13 days after staying home','positiveIncrease -14 days after staying home', 'positive', 'positiveIncrease'])
korrbewallcol

In [ ]:
#nicht benötigte zeilen löschen
korrbewallcol.drop(['zuhause abweichung baseline in prozent'])

Aussage kann hier sein: nachdem mehr Leute zu Hause blieben, ging die Ansteckung langsam zurück

Je schlimmer sich die Krankheit ausbreitet, desto mehr Menschen bleiben zu Hause (müssen zu Hause bleiben) -> die ersten paar Tage nach zu Hause bleiben gibt es noch mehr Ansteckungen, Zahl geht dann langsam zurück

# Zusammenhang Bewegung und Wetter

dennoch wird noch nach einem Zusammenhang zwischen Bewegungsdaten und dem Wetter gesucht

In [ ]:
#wetter und bewegungsdaten zusammenführen
bewwetterny = pd.merge(WetterNY, bewegungny, how='inner', on='date')
bewwetterny.head()

In [ ]:
bewwetterny['date'] = bewwetterny['date'].astype(str)


In [ ]:
bewwetterny.set_index('date', drop=True,inplace=True)


In [ ]:
korrbewwetter=bewwetterny.corr(method ='kendall')
korrbewwetter

In [ ]:
bewkorr=bewwetterny.corr(method ='kendall')
bewkorr['zuhause abweichung baseline in prozent']

keine erkennbare korrelation zwischen wetter und zu Hause bleiben der Menschen

(Wetter schwankt, mal hohe mal tiefe werte aber immer mehr menschen bleiben daheim, umabhängig vom wetter, abhängig von restriktionen)